# LDA Output

>Now that the LDA model has been trained, we can extract the desired output.
>
>Specifically, we will obtain the following:
>* Assigned name for each Genre
>* Word-weighting within each Genre (Top 50 terms)
>* Genre-breakdown per Anime show
>
>These will be written in separate files in the JSON Lines format, for later consumption.

## Load LDA Model

In [1]:
from gensim.models import LdaModel

lda_model = LdaModel.load('lda_model/lda_model')

## Read Text Input

In [2]:
from gensim.corpora.dictionary import Dictionary
from lda_helpers import read_lda_input  # Package with helpers

# Read anime show titles -with text-, for later
title_texts = read_lda_input('lda_input/lda_input.jl', title=True)
texts = [title_text[1] for title_text in title_texts]
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

## Visualize LDA Genres

>Previously, we provided an *eta* matrix to specify a prior assumptions about the genres.
>
>However, the genres produced by the LDA model may not be in the same order as initially specified.
>
>Thus we must observe the produced genres, and then re-assign the Genre names in the appropriate order.

In [3]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
LDAvis_display = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)

# Suppress warning from using pyLDAvis
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

LDAvis_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.131275  0.054746       1        1  22.358316
1      0.079864 -0.320354       2        1  13.221823
2     -0.234813  0.091432       3        1  18.039994
3     -0.008951  0.085365       4        1  14.242926
4      0.327067  0.161125       5        1   7.797969
5     -0.031893 -0.072314       6        1  24.338972, topic_info=        Term         Freq        Total Category  logprob  loglift
255     life  2959.000000  2959.000000  Default  30.0000  30.0000
383   school  2778.000000  2778.000000  Default  29.0000  29.0000
148     team   944.000000   944.000000  Default  28.0000  28.0000
1350    club   856.000000   856.000000  Default  27.0000  27.0000
899   family   998.000000   998.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
80     young   365.323342  1194.104535   Topic6  -4.9600   0.2287
154    world   428.948425  2465.540013   Topic6  -4.7994  -0.3357
255     life   337.561205  2959.142549   Topic6  -5.0390  -0.7578
406    begin   284.335747  1360.982515   Topic6  -5.2106  -0.1527
940     best   252.712286   605.194267   Topic6  -5.3285   0.5398

[542 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
656       1  0.157009  ability
656       2  0.001892  ability
656       3  0.459677  ability
656       4  0.088909  ability
656       5  0.009458  ability
...     ...       ...      ...
80        4  0.071183    young
80        5  0.055272    young
80        6  0.305668    young
762       5  0.997465     zero
2918      2  1.003860   zombie

[927 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6])

## Assign Genre Names

>These match what is shown in the *pyLDAvis* visualization, which is also what *lda_model* holds.
>
>Use the same genres names as provided in *lda_seed.py*

In [4]:
genre_names = [
    'Adventure',      # 1
    'Sports',         # 2
    'Sci-Fi',         # 3
    'Mystery',        # 4
    'Slice of Life',  # 5
    'School'          # 6
]

# Save LDA Results

In [5]:
import json
from os import mkdir

mkdir('lda_output')

## Genre Names

In [6]:
with open('lda_output/genre_names.jl', 'w') as f:
    for i, genre_name in enumerate(genre_names):
        # Write output JSON as newline
        record = {
            'LDA Genre ID': i,
            'LDA Genre Name': genre_name
        }
        line = json.dumps(record)
        f.write('{}\n'.format(line))

## Word Distribution of each Genre (Top 50 Words by Weight)

In [7]:
with open('lda_output/genre_word_weights.jl', 'w') as f:
    for i in range(len(genre_names)):
        genre = lda_model.show_topic(i, topn=50)
        for word, word_weight in genre:
            # Write output JSON as newline
            record = {
                'LDA Genre ID': i,
                'Word': word,
                'Word Weight': float(word_weight)
            }
            line = json.dumps(record)
            f.write('{}\n'.format(line))

## Genre Breakdown of each Anime

In [8]:
with open('lda_output/anime_genre_weights.jl', 'w') as f:
    for i, bow in enumerate(corpus):
        title = title_texts[i][0]
        anime_genres = lda_model.get_document_topics(bow, minimum_probability=0)
        for genre_id, genre_weight in anime_genres:
            # Write output JSON as newline
            record = {
                'Anime Title': title,
                'LDA Genre ID': genre_id,
                'LDA Genre Weight': float(genre_weight)
            }
            line = json.dumps(record)
            f.write('{}\n'.format(line))